# 코드 활용 가이드
- preprocess_config 설정
- preprocess 실행
- config 설정
- eval 실행
- train 실행
- predict 실행

note*
- 클래스를 불러와 실행하는 경우, 작업 경로를 `est_wassup_01/main` 으로 설정하여 진행해주세요.
- import 의 경로가 relative 로 설정되지 않은 경우가 존재합니다.

In [1]:
import os
os.chdir('./main/')

## Preprocess_config 설정
- preprocess_config.py 파일을 수정해주세요.
- 실험에 사용되는 모든 config 파일들은 `est_wassup_01/main/configs` 경로에 위치해 있습니다.

In [9]:
from configs.preprocess_config import config


prerpocess_config = {
  **config,
  "input_data": {
    "train_csv": "./data/origin/train.csv",
    "test_csv": "./data/origin/test.csv",
  },
  # preprocessing 후 저장될 경로를 지정합니다.
  "output_data": {
    "train_feas_csv": "./data/features/train_X.csv",
    "test_feas_csv": "./data/features/test_X.csv",
    "train_target_csv": "./data/features/train_target.csv",
    "y_scaler_save": "./data/features/y_scaler.save",
  },
  # 추가할 외부 데이터들의 경로를 지정합니다.
  "add_data": {
    "add_csv_list": [
      "./data/origin/additionals/bad_cars.csv",  # 0
      "./data/origin/additionals/bad_pedestrian.csv",  # 1
      "./data/origin/additionals/car_per_area_size_dong.csv",  # 2
      "./data/origin/additionals/car_speed.csv",  # 3
      "./data/origin/additionals/car_traffic.csv",  # 4
      "./data/origin/additionals/frozen_road.csv",  # 5
      "./data/origin/additionals/moving_pop_by_age.csv",  # 6
      "./data/origin/additionals/moving_pop_by_days.csv",  # 7
      "./data/origin/additionals/moving_pop_by_time.csv",  # 8
      "./data/origin/additionals/old_pedestrian.csv",  # 9
      "./data/origin/additionals/pedestrian.csv",  # 10
      "./data/origin/additionals/population_mildo_gu_basis.csv",  # 12
      "./data/origin/additionals/rain_mm_monthly.csv",  # 13
      "./data/origin/additionals/road_long_by_dong.csv",  # 14
      "./data/origin/additionals/snow_mean_days_monthly.csv",  # 15
      "./data/origin/additionals/sun_records_monthly_mean.csv",  # 16
      "./data/origin/additionals/truck.csv",  # 17
      "./data/origin/additionals/weather_overall_monthly.csv",  # 18
    ],
  }
}

## Preprocess 실행
- 아래와 같이 Homedata 클래스를 불러와서 실행할 수 있습니다.
- 혹은 `est_wassup_01/main` 경로에서 `python preprocess.py` 를 통해 실행하실 수 있습니다.

In [3]:
from datasets import HomeData
from utils import create_path_if_not_exists
import joblib

input_data = prerpocess_config.get('input_data')
add_data = prerpocess_config.get('add_data')
options = prerpocess_config.get('options')
output_data = prerpocess_config.get('output_data')

trn_X, trn_y, tst_X, y_scaler = HomeData(
  **input_data,
  **add_data,
  **options
).preprocess()

for k,v in output_data.items():
  create_path_if_not_exists(v, True, '/')
trn_X.to_csv(output_data.get('train_feas_csv'))
tst_X.to_csv(output_data.get('test_feas_csv'))
trn_y.to_csv(output_data.get('train_target_csv'))
joblib.dump(y_scaler, output_data.get('y_scaler_save'))

(39609, 156)
(10963, 156)


['./data/features/y_scaler.save']

## Config 설정
- config.py 파일을 수정해주세요.
- 실험에 사용되는 모든 config 파일들은 `est_wassup_01/main/configs` 경로에 위치해 있습니다.

In [10]:
from configs.config import config

EXPERIMENT_NAME = 'guide-note'

config = {
  **config,
  "input_files": {
    "X_csv": "./data/features/train_X.csv",
    "y_csv": "./data/features/train_target.csv",
    "tst_csv": "./data/features/test_X.csv",
  },
  "output_files": {
    "output_train_loss": f"./histories/{EXPERIMENT_NAME}/trn_loss.csv",
    "output_train": f"./histories/{EXPERIMENT_NAME}/model.pth",
    "output_eval": f"./histories/{EXPERIMENT_NAME}/eval.csv",
    "output_pred": f"./histories/{EXPERIMENT_NAME}/pred.csv",
  },
}

## Train 실행
- 아래와 같이 TrainMaker / Train 클래스를 불러와서 실행할 수 있습니다.
- 혹은 `est_wassup_01/main` 경로에서 `python train.py` 를 통해 실행하실 수 있습니다.

In [7]:
from torch import manual_seed
from trains import Train
from makers import TrainMaker


manual_seed(2023)
train_maker = TrainMaker(**config)
Train.run(train_maker)

  0%|          | 0/50 [00:00<?, ?it/s]

# Eval 실행
- 아래와 같이 EvalMaker / KFoldCV 클래스를 불러와서 실행할 수 있습니다.
- 혹은 `est_wassup_01/main` 경로에서 `python eval.py` 를 통해 실행하실 수 있습니다.

In [8]:
from makers import EvalMaker
from model_selection import KFoldCV, HoldoutWithTestCSV
import pandas as pd
from utils import create_path_if_not_exists


eval_maker = EvalMaker(**config)
cv = KFoldCV(eval_maker)
# cv = HoldoutWithTestCSV(eval_maker)
res = cv()
res = pd.concat([res, res.iloc[:,:2].apply(['mean'])])
print(res)
create_path_if_not_exists(eval_maker.output_eval)
res.to_csv(eval_maker.output_eval)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

        trn_r2    val_r2                                        trn_history  \
0     0.023054  0.029884  {'trn_mae': [0.05293317139148712, 0.0320810712...   
1     0.022644  0.022822  {'trn_mae': [0.05559570714831352, 0.0323258489...   
2     0.025091  0.028350  {'trn_mae': [0.055722445249557495, 0.032495725...   
3     0.022227  0.028448  {'trn_mae': [0.05972567945718765, 0.0338713079...   
4     0.023142  0.019013  {'trn_mae': [0.054147712886333466, 0.033395677...   
mean  0.023232  0.025703                                                NaN   

                                            val_history  
0     {'val_mae': [0.03101440519094467, 0.0294005479...  
1     {'val_mae': [0.034833211451768875, 0.030823865...  
2     {'val_mae': [0.03943844139575958, 0.0291497204...  
3     {'val_mae': [0.03055543452501297, 0.0325411967...  
4     {'val_mae': [0.030459867790341377, 0.027156822...  
mean                                                NaN  


## Predict 실행
- Predict 의 경우, preprocess_config 와 config 둘 모두를 활용합니다.
- 아래와 같이 Predict 클래스를 불러와서 실행할 수 있습니다.
- 혹은 `est_wassup_01/main` 경로에서 `python predict.py` 를 통해 실행하실 수 있습니다.

In [11]:
from inferences import Predict


options = prerpocess_config.get('options')
output_csv = prerpocess_config.get('output_data')
Predict(
  index_col=options.get('index_col'),
  target_cols=options.get('target_cols'),
  target_drop_col=options.get('target_drop_col'),
  y_scaler_save=output_csv.get('y_scaler_save'),
  y_scaler=options.get('y_scaler'),
  **config
).run()